In [1]:
import os
import time
import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys


In [2]:
# Set Chrome options for headless mode
chrome_options = Options()
# Jalan tanpa UI
chrome_options.add_argument("--headless")
download_dir = os.path.abspath("D:\\Projects\\Appsenin\\backend\\dump\\output")

prefs = {
    "download.default_directory": download_dir,  # Set lokasi folder unduhan
    "download.prompt_for_download": False,  # Jangan tampilkan dialog unduhan
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True,
    "safebrowsing.disable_download_protection": True  # Mencegah pemblokiran file
}

chrome_options.add_experimental_option("prefs", prefs)

chrome_options.add_argument("--window-size=1920,1080")
# User Agent
chrome_options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120 Safari/537.36"
)

In [3]:
# tanggal
date_now = datetime.datetime.now().date()
date_now.day


12

In [4]:
def renaming_file(download_dir):
    # Rename file
    for filename in os.listdir(download_dir):
        if filename.endswith(".xlsx"):
            os.rename(
                os.path.join(download_dir, filename),
                os.path.join(download_dir, f"pegawai_{date_now}.xlsx"),
            )

In [5]:
# fungsi get data
def process_noncheckout(driver, day):
    wait = WebDriverWait(driver, 5)

    # select Unit Level
    unit_level_input = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[2]/div/div[2]/div/div[1]/div/div[2]/div/input")))
    unit_level_input.clear()
    unit_level_input.send_keys("PLN")
    unit_level_input.click()
    time.sleep(2)
    # tunggu popup muncul
    wait.until(EC.visibility_of_element_located((By.XPATH,"//div/div/div[2]/div/div[2]/div/div[1]/div/div[2]/div/div")))
    # pilih opsi unit level
    selected_unit_level = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[2]/div/div[2]/div/div[1]/div/div[2]/div/div/div/ul/li[1]")))
    selected_unit_level.click()

    # Select hari awal
    start_day = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[placeholder='Tanggal Mulai']")))
    start_day.click()
    time.sleep(1)
    today_date = day.today().strftime("%dd-%mm-%YYYY")

    # Masukkan tanggal
    start_day.send_keys(today_date)
    start_day.send_keys(Keys.RETURN)
    time.sleep(1)
    
    # Select hari akhir
    end_day = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[placeholder='Tanggal Selesai']")))
    end_day.click()
    time.sleep(1)

     # Masukkan tanggal
    end_day.send_keys(today_date)
    end_day.send_keys(Keys.RETURN)
    time.sleep(1)

    # Klik Button Cari
    search_button = wait.until(EC.element_to_be_clickable((By.XPATH, ".//*/div/div[2]/div/div/div/div[1]/div/div[7]/button")))
    search_button.click()

    time.sleep(5) 

    download_button = wait.until(EC.element_to_be_clickable((By.XPATH, ".//*/div/div[2]/div/div/div/div[2]/div/div[1]/img[2]") ))
    download_button.click()
    time.sleep(5)
    renaming_file(download_dir)

In [6]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

url = os.getenv("ABSEN_LINK")
driver.get(url)

username_input = WebDriverWait(driver, 30).until(
    EC.visibility_of_element_located((By.XPATH, ".//*/form/div[3]/div/input"))
)
time.sleep(1)
username_input.send_keys(os.getenv("USER_NAME_LINK"))

password_input = WebDriverWait(driver, 30).until(
	EC.visibility_of_element_located((By.XPATH, ".//*/form/div[4]/div/div/input"))
)
time.sleep(1)
password_input.send_keys(os.getenv("PASSWORD_LINK"))

login_button = WebDriverWait(driver, 30).until(
	EC.visibility_of_element_located((By.XPATH, ".//*/form/div[5]/div/button"))
)
time.sleep(1)
login_button.click()

# Wait for the login to complete
time.sleep(10)
# klik menu absen kehadiran
kedisplinan_menu = WebDriverWait(driver, 30).until(
	EC.visibility_of_element_located((By.XPATH, "//*/div[1]/div[1]/ul/div[3]/div[1]/div"))
)
time.sleep(2)
kedisplinan_menu.click()


table_data = process_noncheckout(driver, datetime.datetime)
driver.quit()

Process Generate API

In [7]:

DataFrame = pd.read_excel(f"{download_dir}\\pegawai_{date_now}.xlsx")
name_list = DataFrame[(DataFrame['Absen Keluar'].isna()) & (DataFrame['Absen Masuk'].notna())]['Nama'].tolist()
print(name_list)

['Achmad Syachrul Alim', 'AFRIZAL FIRMANSYAH', 'Astri Nailah Aurelia', 'BEY ARYO ABIANTORO', 'BRIAN ADITIAYUDA', 'CATTLEYA CAELI LUIS', 'CHOIRUNNISA', 'DHAMAR SUMARWAN', 'Dian Lesmana', 'DIAN THERESIA', 'DWI PUTRI BONITA', 'EDY SUSILO', 'ERDIYANSYAH RIFWAN', 'FAIZAL LUBIS', 'Fata Sabita Ala Dini', 'Fatahillah Dian Pratomo', 'FENDY PURNOMO', 'Gita Diaz Pangesti', 'HASTI TRIANA PUTRI', 'HERRY ISMAIL', 'I Gst Ayu Nyn Trisna Wahyuni', 'IBRAHIM MANOREK', 'Irfa Ramadhanti', "JA'FAHRUDIN", 'KURNIA SAFITRI RAMADHAN', 'Laila Alfi Syah', 'LILIK NURHOLIDAH', 'M Fikri Damar Muchtarom', 'M. NOVAL KURNIAWAN', 'Mina Celia Muhidin', 'MOHAMMAD RIZZAL FIRDAUS ALMUGNI', 'Muhammad Afwan Fajri', 'Muhammad Alghany Jagad Aryasatya', 'Muhammad Arief Hidayattullah', 'MUHAMMAD BAGAS PRIAMBODO', 'Muhammad Candra Firmansyah', 'MUHAMMAD SULTAN ARIF', 'MUKHAMMAD THAUFIQ BAYU AJI', 'NANDA DWI CHRISTANTO', 'NIKEN RAHMADANI', 'NOVERRY AMBO', 'NURKHOLIS ARI SUGIARTO', 'OKSYA AFIFAH', 'Putri Salsabila Dewi', 'RATU TRIA'

In [8]:
non_checkout_data = {}
def generate_api_payload(name_list):
    for name in name_list:
        non_checkout_data[name] = {
            "status": "non-checkout"
        }

print(generate_api_payload(name_list))

print(datetime.datetime.now().time(), non_checkout_data)

None
14:54:01.790604 {'Achmad Syachrul Alim': {'status': 'non-checkout'}, 'AFRIZAL FIRMANSYAH': {'status': 'non-checkout'}, 'Astri Nailah Aurelia': {'status': 'non-checkout'}, 'BEY ARYO ABIANTORO': {'status': 'non-checkout'}, 'BRIAN ADITIAYUDA': {'status': 'non-checkout'}, 'CATTLEYA CAELI LUIS': {'status': 'non-checkout'}, 'CHOIRUNNISA': {'status': 'non-checkout'}, 'DHAMAR SUMARWAN': {'status': 'non-checkout'}, 'Dian Lesmana': {'status': 'non-checkout'}, 'DIAN THERESIA': {'status': 'non-checkout'}, 'DWI PUTRI BONITA': {'status': 'non-checkout'}, 'EDY SUSILO': {'status': 'non-checkout'}, 'ERDIYANSYAH RIFWAN': {'status': 'non-checkout'}, 'FAIZAL LUBIS': {'status': 'non-checkout'}, 'Fata Sabita Ala Dini': {'status': 'non-checkout'}, 'Fatahillah Dian Pratomo': {'status': 'non-checkout'}, 'FENDY PURNOMO': {'status': 'non-checkout'}, 'Gita Diaz Pangesti': {'status': 'non-checkout'}, 'HASTI TRIANA PUTRI': {'status': 'non-checkout'}, 'HERRY ISMAIL': {'status': 'non-checkout'}, 'I Gst Ayu Nyn T